In [3]:
import tables_configs
from pyspark.sql import DataFrame as SparkDataFrame
from pyspark.sql.types import StructType, IntegerType,StringType,FloatType,DecimalType
from pyspark.sql.functions import from_json, col, regexp_replace

In [ ]:
STORAGE_ACCOUNT_NAME = "azuredatalakespuertaf"
RAW_PATH = f"wasbs://raw@{STORAGE_ACCOUNT_NAME}.blob.core.windows.net"
MOUNT_POINT_RAW = "/mnt/raw/"
RAW_CHECKPOINT_LOCATION = "/mnt/raw/{}/check/"
DATABRICKS_SECRET_SCOPE = "accessdatalake"
AZURE_SECRET_PATH = f"fs.azure.account.key.{STORAGE_ACCOUNT_NAME}.blob.core.windows.net"
AZURE_SECRET_NAME = "datalakeaccess"
ENTITIES = ["customer","date","product","reseller","sales","sales_order","sales_territory"]
TOPICS = ["prueba.customer","prueba.date","prueba.product","prueba.reseller","prueba.sales","prueba.sales-order","prueba.sales-territory"]
KAFKA_SERVER = "20.49.8.224:9092"
SAVE_DATA_FORMAT = "delta"
SAVE_DATA_MODE = "append"
OUT_PATHS = {entity:MOUNT_POINT_RAW+entity+"/" for entity in ENTITIES}
print(OUT_PATHS)

In [ ]:
#configurando y montando puntos de montaje raw DataBricks - Azure
dbutils.fs.mount(
    source = RAW_PATH,
    mount_point = MOUNT_POINT_RAW,
    extra_configs = {AZURE_SECRET_PATH:dbutils.secrets.get(scope=DATABRICKS_SECRET_SCOPE, key=AZURE_SECRET_NAME)}
)

In [ ]:
schema_customer = StructType()\
                  .add("CustomerKey",IntegerType())\
                  .add("CustomerID",StringType())\
                  .add("Customer",StringType())\
                  .add("City",StringType())\
                  .add("StateProvince",StringType())\
                  .add("CountryRegion",StringType())\
                  .add("PostalCode",StringType())


schema_date = StructType()\
              .add("DateKey",IntegerType())\
              .add("Date",StringType())\
              .add("FiscalYear",StringType())\
              .add("FiscalQuarter",StringType())\
              .add("Month",StringType())\
              .add("FullDate",StringType())\
              .add("MonthKey",IntegerType())

schema_product = StructType()\
                 .add("ProductKey",IntegerType())\
                 .add("SKU",StringType())\
                 .add("Product",StringType())\
                 .add("StandardCost",FloatType())\
                 .add("Color",StringType())\
                 .add("ListPrice",FloatType())\
                 .add("Model",StringType())\
                 .add("Subcategory",StringType())\
                 .add("Category",StringType())

schema_reseller = StructType()\
                 .add("ResellerKey",IntegerType())\
                 .add("ResellerID",StringType())\
                 .add("BusinessType",StringType())\
                 .add("Reseller",StringType())\
                 .add("City",StringType())\
                 .add("StateProvince",StringType())\
                 .add("CountryRegion",StringType())\
                 .add("PostalCode",StringType())

schema_sales = StructType()\
               .add("SalesOrderLineKey",IntegerType())\
               .add("ResellerKey",IntegerType())\
               .add("CustomerKey",IntegerType())\
               .add("ProductKey",IntegerType())\
               .add("OrderDateKey",IntegerType())\
               .add("DueDateKey",IntegerType())\
               .add("ShipDateKey",IntegerType())\
               .add("SalesTerritoryKey",IntegerType())\
               .add("OrderQuantity",IntegerType())\
               .add("UnitPrice",FloatType())\
               .add("ExtendedAmount",FloatType())\
               .add("UnitPriceDiscountPct",DecimalType(5,2))\
               .add("ProductStandardCost",FloatType())\
               .add("TotalProductCost",FloatType())\
               .add("SalesAmount",FloatType())

schema_sales_order = StructType()\
                 .add("Channel",StringType())\
                 .add("SalesOrderLineKey",IntegerType())\
                 .add("SalesOrder",StringType())\
                 .add("SalesOrderLine",IntegerType())

schema_sales_territory = StructType()\
                         .add("SalesTerritoryKey",IntegerType())\
                         .add("Region",StringType())\
                         .add("Country",StringType())\
                         .add("Group",StringType())
                         
schemas = [schema_customer,schema_date,schema_product,schema_reseller,schema_sales,schema_sales_order,schema_sales_territory]

In [ ]:
def read_kafka_topic(kafka_server:str, topic_name:str) -> SparkDataFrame:
     """Leer topico de kafka del servidor y convertirlo a streaming de datos. 

     Args:
         kafka_server (str): Direccion IP y puerto para acceder a la maquina virtual con kafka. 
         topic_name (str): Nombre del topico de kafka del cual se extraeran los datos.

     Returns:
         SparkDataFrame: Streamin de datos del topico 
     """
     stream_df = spark.readStream\
         .format("kafka")\
         .option("kafka.bootstrap.servers", kafka_server)\
         .option("subscribe",topic_name)\
         .option("startingOffsets","earliest")\
         .option("failOnDataLoss","false")\
         .load()
     return stream_df

In [ ]:
def get_data(stream_df:SparkDataFrame, schema:StructType) -> SparkDataFrame:
    """Se obtiene los datos base del topico aplicandole el esquema provisto.

    Args: 
            df(SparkDataFrame): DataFrame de Spark base del topico.
            Schema(StruckType): Eschema de datos esperados en la trama.
    Returns: 
            df(SparkDataFrame): DataFrame de Spark con los datos de la trama parseados según el esquema definido.
  """
    df = stream_df.select(from_json(col("value").cast(StringType()),schema).alias("data"))
    df = df.select("data.*")
    return df

In [1]:

def get_columns_2_transform(table_name:str, columns_2_transform_type:str) -> list:
    """Obtener de las configuaraciones globales de la tabla (ConfigQuind) las columnas
    con tipos de datos especiales a imputar. 

    Args:
        table_name (str): Nombre de la tabla que posee un tipo de dato especifico a inputar
        columns_2_transform_type (str): Tipo de dato especifico a imputar

    Returns:
        list: Lista de tuplas que contiene el nombre de la columns y su tipo especificado
              en las configuarciones globales.

    Example:
    >>> get_columns_2_transform("sales","price")
            [('ExtendedAmount', 'price'),
             ('UnitPrice', 'price'),
             ('TotalProductCost', 'price'),
             ('SalesAmount', 'price')]
    >>> get_columns_2_transform("date","price")
            []
    """
    # me aseguro de que la tabla tenga configurada variables globales
    try:
        tables_configs.global_confs[table_name]
    except KeyError:
        #en caso de que la tabla no tenga configuraciones especificadas se retorna lista vacia
        return []
    assert columns_2_transform_type in tables_configs.columns_types_2_transform, f"{columns_2_transform_type} type not in tables_config allowed types"
    return list(filter(lambda x:x[1] == columns_2_transform_type,tables_configs.global_confs[table_name].items()))

In [ ]:

def transform_prices_2_numeric(df:SparkDataFrame, table_name:str) -> SparkDataFrame:
    """Imputar datos de las columnas de la tabla 'table_name' que tienen datos de tipo 'price' 
    a datos flotantes. 

    Args:
        df (SparkDataFrame): DataFrame de Spark que contiene los datos leidos del topico de Kafka.
        table_name (str): Nombre de la tabla que contiene las columnas con tipos de datos
        'price' a imputar.

    Returns:
        SparkDataFrame: DataFrame de Spark con columnas que contiene tipos de datos 'price' imputadas.
    
    Example:
    >>> df_input.show()
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
        |SalesOrderLineKey|ResellerKey|CustomerKey|ProductKey|OrderDateKey|DueDateKey|ShipDateKey|SalesTerritoryKey|OrderQuantity|UnitPrice|ExtendedAmount|UnitPriceDiscountPct|ProductStandardCost|TotalProductCost|SalesAmount|
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
        |         43659001|        676|         -1|       349|    20170702|  20170712|   20170709|                5|            1|$2.024,99|     $2.024,99|               0,00%|          $1.898,09|       $1.898,09|  $2.024,99|
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
    
    >>> transform_prices_2_numeric(df_iput,"sales")
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
        |SalesOrderLineKey|ResellerKey|CustomerKey|ProductKey|OrderDateKey|DueDateKey|ShipDateKey|SalesTerritoryKey|OrderQuantity|UnitPrice|ExtendedAmount|UnitPriceDiscountPct|ProductStandardCost|TotalProductCost|SalesAmount|
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
        |         43659001|        676|         -1|       349|    20170702|  20170712|   20170709|                5|            1|  2024.99|       2024.99|               0,00%|            1898.09|         1898.09|    2024.99|
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
    """
    if len(get_columns_2_transform(table_name,"price")) == 0:
        return df
    
    columns_2_transform = list(map(
        lambda x: x[0],
        get_columns_2_transform(table_name,"price")
    ))
    # checkeo que las columnas retornadas si esten en el dataframe
    assert len(set(columns_2_transform) - set(df.columns)) == 0, f"Mismatch between df columns and global {table_name} columns" 
    for column in columns_2_transform:
        df = df.withColumn(
            column,
            regexp_replace(
                regexp_replace(col(column), "[$.]", ""), "[,]","."
            ).alias(column).cast("float")
        )
    return df

In [ ]:
def transform_percentages_2_numeric(df, table_name):
    """Imputar datos de las columnas de la tabla 'table_name' que tienen datos de tipo 'percentage' 
    a datos flotantes.  


    Args:
        df (SparkDataFrame): DataFrame de Spark que contiene los datos leidos del topico de Kafka.
        table_name (str): Nombre de la tabla que contiene las columnas con tipos de datos
        'percentage' a imputar.

    Returns:
        SparkDataFrame: DataFrame de Spark con columnas que contiene tipos de datos 'percentage' imputadas.

    Example:
    >>> df_input.show()
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
        |SalesOrderLineKey|ResellerKey|CustomerKey|ProductKey|OrderDateKey|DueDateKey|ShipDateKey|SalesTerritoryKey|OrderQuantity|UnitPrice|ExtendedAmount|UnitPriceDiscountPct|ProductStandardCost|TotalProductCost|SalesAmount|
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
        |         43659001|        676|         -1|       349|    20170702|  20170712|   20170709|                5|            1|$2.024,99|     $2.024,99|               0,00%|          $1.898,09|       $1.898,09|  $2.024,99|
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
    
    >>> transform_percentages_2_numeric(df_input, "sales")
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
        |SalesOrderLineKey|ResellerKey|CustomerKey|ProductKey|OrderDateKey|DueDateKey|ShipDateKey|SalesTerritoryKey|OrderQuantity|UnitPrice|ExtendedAmount|UnitPriceDiscountPct|ProductStandardCost|TotalProductCost|SalesAmount|
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
        |         43659001|        676|         -1|       349|    20170702|  20170712|   20170709|                5|            1|$2.024,99|     $2.024,99|                 0.0|          $1.898,09|       $1.898,09|  $2.024,99|
        +-----------------+-----------+-----------+----------+------------+----------+-----------+-----------------+-------------+---------+--------------+--------------------+-------------------+----------------+-----------+
    """
    if len(get_columns_2_transform(table_name,"percentage")) == 0:
        return df
    
    columns_2_transform = list(map(
        lambda x: x[0],
        get_columns_2_transform(table_name,"percentage")
    ))
    # checkeo que las columnas retornadas si esten en el dataframe
    assert len(set(columns_2_transform) - set(df.columns)) == 0, f"Mismatch between df columns and global {table_name} columns"
    for column in columns_2_transform:
        df = df.withColumn(
            column,
            regexp_replace(
                regexp_replace(col(column), "[%]", ""), "[,]","."
            ).alias(column).cast("float")
        )
    return df 

In [ ]:
def save_data(data_df:SparkDataFrame,format_:str,out_path:str,mode:str, check_path:str):
    """
    Funcion principal que define todo el proceso de streaming: carga la trama base, genera la funcion de procesamiento y las rutas de checkpoint. 
    Args: 
        df(SparkDataFrame): DataFrame de Spark
        format(str): Formato de Almacenamiento: delta, parquet, etc
        output_path(str): Ruta de almacenamiento
        query_name(str): Nombre del Query 
        mode(str): Modo de almacenamiento
        check_path(str): Ruta de Almacenamiento del checkpoint
        
    Returns 
        Streaming 
  """
    return data_df.writeStream\
                  .format(format_)\
                  .option("path",out_path)\
                  .outputMode(mode)\
                  .option("checkpointLocation",check_path)\
                  .option("mergeSchema","true")\
                  .trigger(processingTime="30 seconds")

In [ ]:
def main(table_name:str, kafka_server:str, topic_name:str, schema:StructType,format_:str, out_path:str,mode:str, check_path:str):
    """
    Funcion principal que define todo el proceso de streaming: carga la trama base, la parsea, ejecuta la logica de negocio y escribe los datos resultado. 
    Args: 
        table_name(str): Nombre de la tabla 
        kafka_server(str): Direccion IP del Broker del servicio de Kafka.
        topic_name(str): Topico de kafka 
        Schema(StruckType): Eschema de datos esperados en la trama de entrada.
        format_(str): Formato de Almacenamiento: delta, parquet, etc
        out_path(str): Ruta de almacenamiento
        mode(str): Modo de almacenamiento
        check_path(str): Ruta de Almacenamiento del checkpoint
    Returns 
        Streaming 
  """
    kafka_data = read_kafka_topic(
        kafka_server=kafka_server,
        topic_name=topic_name
    )
    parsed_data = get_data(
        stream_df=kafka_data,
        schema=schema
    )
    
    clean_prices_df = transform_prices_2_numeric(
        df = parsed_data,
        table_name=table_name,
    )
    clean_percentages_df = transform_percentages_2_numeric(
        df=clean_prices_df,
        table_name=table_name,
    )
    saved_data = save_data(
        data_df=clean_percentages_df,
        format_=format_,
        out_path=out_path,
        mode = mode,
        check_path=check_path
    )
    return saved_data

In [ ]:
customer_df = main(
        table_name = "customer",
        kafka_server= KAFKA_SERVER,
        topic_name="prueba.customer",
        schema = schema_customer,
        format_= SAVE_DATA_FORMAT,
        out_path= OUT_PATHS["customer"],
        mode = SAVE_DATA_MODE,
        check_path=RAW_CHECKPOINT_LOCATION.format("customer")
    )
customer_df.start()

In [ ]:
date_df = main(
        table_name="date",
        kafka_server= KAFKA_SERVER,
        topic_name="prueba.date",
        schema = schema_customer,
        format_= SAVE_DATA_FORMAT,
        out_path= OUT_PATHS["date"],
        mode = SAVE_DATA_MODE,
        check_path=RAW_CHECKPOINT_LOCATION.format("date")
    )
date_df.start()


In [ ]:
sales_df = main(
        table_name="sales",
        kafka_server= KAFKA_SERVER,
        topic_name="prueba.sales",
        schema = schema_customer,
        format_= SAVE_DATA_FORMAT,
        out_path= OUT_PATHS["sales"],
        mode = SAVE_DATA_MODE,
        check_path=RAW_CHECKPOINT_LOCATION.format("sales")
    )
sales_df.start()

In [ ]:
#Para detener los procesos de streaming
for s in spark.streams.active:
      s.stop()